In [ ]:
# dependencies for tracker - 
# !pip install filterpy
# !pip install scikit-image
# !pip install lap

In [1]:
#tracking id - 
import sort # sort.py file containing tracker code
from sort import *
from ultralytics import YOLO
import cv2
import cvzone
import time
import math

In [2]:
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

In [30]:
# use canva to design your mask ,create design, load video in your design , press r to create boxes and mask , finally downlaod it

In [5]:
cap = cv2.VideoCapture(r"C:\Users\Dell\Downloads\Analysis\project9-Object_detection\vid.mp4") # path
model = YOLO(r'C:\Users\Dell\Downloads\Analysis\project9-Object_detection\yolov8n.pt')
mask = cv2.imread('yy.png')
# resize mask -
mask = cv2.resize(mask, (int(cap.get(3)), int(cap.get(4))))
while True:
    success,img = cap.read()
    # for masked background
    imgRegion = cv2.bitwise_and(img,mask)
    result = model(imgRegion ,stream=True)
    for r in result:
        boxes = r.boxes
        for box in boxes:
            #x1,y1,x2,y2 = box.xyxy[0]
            #x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
            #cv2.rectangle(img(x1,y1),(x2,y2),(0,200,0),3) # img points,color,thickness of box
            
            # for bounding box (using cvzone) - 
            x1,y1,x2,y2 = box.xyxy[0]
            x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
            w,h = x2-x1,y2-y1
            #cvzone.cornerRect(img,(x1,y1,w,h))
            # for confidence level - 
            conf = math.ceil((box.conf[0]*100))/100
            # for class - 
            cls = int(box.cls[0])
            currclass = classNames[cls]
            if currclass =="car" or currclass == 'truck' and conf > 0.4:
                cvzone.putTextRect(img,f'{currclass} {conf}',(max(0,x1),max(25,y1)),
                                  scale=0.6,thickness=1,offset=3)# ,scale=0.7,thickness=1
                cvzone.cornerRect(img,(x1,y1,w,h),l=6,t=3,rt=1)
                
                
            
             
            
            
    cv2.imshow('Image',img)
    cv2.imshow('ImageRegion',imgRegion)
    # Wait for a key press, and break the loop if any key is pressed
    key = cv2.waitKey(1) & 0xFF
    if key != 255:  # Check if any key is pressed (not equal to 255)
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


0: 384x640 6 cars, 147.7ms
Speed: 3.5ms preprocess, 147.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 237.3ms
Speed: 27.1ms preprocess, 237.3ms inference, 16.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 171.4ms
Speed: 1.0ms preprocess, 171.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 149.0ms
Speed: 3.1ms preprocess, 149.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 156.2ms
Speed: 3.1ms preprocess, 156.2ms inference, 12.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 140.0ms
Speed: 3.9ms preprocess, 140.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 140.2ms
Speed: 3.0ms preprocess, 140.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 166.1ms
Speed: 2.9ms preprocess, 166.1ms inference, 0.0ms postprocess per image at shape (1, 3,

In [3]:
cap = cv2.VideoCapture(r"C:\Users\Dell\Downloads\Analysis\project9-Object_detection\vid.mp4") # path
model = YOLO(r'C:\Users\Dell\Downloads\Analysis\project9-Object_detection\yolov8n.pt')
mask = cv2.imread('yy.png')
# resize mask -
mask = cv2.resize(mask, (int(cap.get(3)), int(cap.get(4))))
tracker = Sort(max_age=20,min_hits=3,iou_threshold=0.3)
limits = [450,480,850,480]
totalCount = []
while True:
    success, img = cap.read()
    imgRegion = cv2.bitwise_and(img, mask)
    results = model(imgRegion, stream=True)
 
    detections = np.empty((0, 5))
 
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
            w, h = x2 - x1, y2 - y1
 
            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Class Name
            cls = int(box.cls[0])
            currentClass = classNames[cls]
 
            if currentClass == "car" or currentClass == "truck" or currentClass == "bus" \
                    or currentClass == "motorbike" and conf > 0.3:
                # cvzone.putTextRect(img, f'{currentClass} {conf}', (max(0, x1), max(35, y1)),
                #                    scale=0.6, thickness=1, offset=3)
                # cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=5)
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))
 
    resultsTracker = tracker.update(detections)
 
    cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 0, 255), 5)
    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        print(result)
        w, h = x2 - x1, y2 - y1
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
        cvzone.putTextRect(img, f' {int(id)}', (max(0, x1), max(35, y1)),
                           scale=2, thickness=3, offset=10)
        # circle denoting centre point , when these touch the line then only count
        cx, cy = x1 + w // 2, y1 + h // 2
        cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)
        # if point detected in this region , count
        if limits[0] < cx < limits[2] and limits[1] - 15 < cy < limits[1] + 15:
            if totalCount.count(id) == 0:
                totalCount.append(id)
                cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 255, 0), 5)
 
    # cvzone.putTextRect(img, f' Count: {len(totalCount)}', (50, 50))
    cv2.putText(img,f'count : {str(len(totalCount))}',(255,100),cv2.FONT_HERSHEY_PLAIN,4,(50,50,255),8)
 
    cv2.imshow("Image", img)
    # cv2.imshow("ImageRegion", imgRegion)
    key = cv2.waitKey(1) & 0xFF
    if key != 255:  # Check if any key is pressed (not equal to 255)
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


0: 384x640 6 cars, 283.9ms
Speed: 9.0ms preprocess, 283.9ms inference, 21.1ms postprocess per image at shape (1, 3, 384, 640)



[        695         342         801         410           6]
[        666         244         734         296           5]
[        505         273         589         343           4]
[        661         301         744         382           3]
[        472         302         568         380           2]
[        694         316         803         409           1]


0: 384x640 5 cars, 230.7ms
Speed: 5.8ms preprocess, 230.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 158.9ms


[     665.76      245.19      732.24      296.81           5]
[     505.43      275.65      588.57      343.35           4]
[        662         303         744         383           3]
[     473.15      303.88      568.85      381.12           2]
[     694.86      317.12      804.14      410.88           1]


Speed: 1.6ms preprocess, 158.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



...

[     515.48      245.35      604.91      310.85          22]
[     653.06      246.95      757.42      341.27          21]
[     661.18      312.13      770.41      401.33          20]
[     744.69      461.81      922.59      603.31          13]
[     394.53       443.9      542.65      570.55          12]


0: 384x640 5 cars, 183.1ms
Speed: 2.5ms preprocess, 183.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 182.3ms


[     514.29      246.04      604.24      312.21          22]
[     653.72       247.6      757.21      342.12          21]
[      661.5       313.3      770.61      403.15          20]
[     746.33       466.9      926.39      610.55          13]
[     392.25      447.92      541.54      576.06          12]


Speed: 3.8ms preprocess, 182.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 162.6ms
Speed: 1.1ms preprocess, 162.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


[     513.76      246.37      604.09      313.29          22]
[     654.55       247.6      757.13      341.99          21]
[      661.9      313.99      770.39      404.84          20]
[     747.19      471.14      928.79      616.61          13]
[     390.76      451.21      541.16      580.71          12]
[     513.51      246.54      604.11      314.28          22]
[     655.93      247.62      757.28      341.91          21]
[     662.81      314.69      770.83      406.28          20]
[     748.91       476.2      934.09      623.04          13]
[     388.63      453.96      540.68      585.29          12]


In [6]:
# for video 2 - 
cap = cv2.VideoCapture(r"C:\Users\Dell\Downloads\Analysis\project9-Object_detection\vid1.mp4") # path
model = YOLO(r'C:\Users\Dell\Downloads\Analysis\project9-Object_detection\yolov8n.pt')
mask = cv2.imread('xx.png')
# resize mask -
mask = cv2.resize(mask, (int(cap.get(3)), int(cap.get(4))))
tracker = Sort(max_age=20,min_hits=3,iou_threshold=0.3)
limits = [170,360,400,360]
totalCount = []
while True:
    success, img = cap.read()
    imgRegion = cv2.bitwise_and(img, mask)
    results = model(imgRegion, stream=True)
 
    detections = np.empty((0, 5))
 
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
            w, h = x2 - x1, y2 - y1
 
            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Class Name
            cls = int(box.cls[0])
            currentClass = classNames[cls]
 
            if currentClass == "car" or currentClass == "truck" or currentClass == "bus" \
                    or currentClass == "motorbike" and conf > 0.3:
                # cvzone.putTextRect(img, f'{currentClass} {conf}', (max(0, x1), max(35, y1)),
                #                    scale=0.6, thickness=1, offset=3)
                # cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=5)
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))
 
    resultsTracker = tracker.update(detections)
 
    cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 0, 255), 5)
    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        print(result)
        w, h = x2 - x1, y2 - y1
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
        cvzone.putTextRect(img, f' {int(id)}', (max(0, x1), max(35, y1)),
                           scale=2, thickness=3, offset=10)
        # circle denoting centre point , when these touch the line then only count
        cx, cy = x1 + w // 2, y1 + h // 2
        cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)
        # if point detected in this region , count
        if limits[0] < cx < limits[2] and limits[1] - 15 < cy < limits[1] + 15:
            if totalCount.count(id) == 0:
                totalCount.append(id)
                cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 255, 0), 5)
 
    # cvzone.putTextRect(img, f' Count: {len(totalCount)}', (50, 50))
    cv2.putText(img,f'count : {str(len(totalCount))}',(255,100),cv2.FONT_HERSHEY_PLAIN,4,(50,50,255),8)
 
    cv2.imshow("Image", img)
    # cv2.imshow("ImageRegion", imgRegion)
    key = cv2.waitKey(1) & 0xFF
    if key != 255:  # Check if any key is pressed (not equal to 255)
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


0: 384x640 6 cars, 1 truck, 236.4ms
Speed: 5.9ms preprocess, 236.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



[        339         251         371         287          76]
[        330         290         360         310          75]
[        377         280         402         301          74]
[        306         277         332         294          73]
[        267         299         294         319          72]
[        356         300         390         329          71]
[        284         323         322         355          70]


0: 384x640 7 cars, 1 truck, 300.7ms
Speed: 29.4ms preprocess, 300.7ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)



[        376         279         404         307          77]
[     336.04      249.96      370.96      289.04          76]
[     330.18      289.89      362.82      311.11          75]
[     377.74      280.23      402.26      301.77          74]
[     306.12      276.92      332.88      294.08          73]
[     266.84      299.12      294.16      319.88          72]
[      356.4      301.68       390.6      329.32          71]
[     281.86      323.12      320.14      355.88          70]


0: 384x640 7 cars, 1 truck, 270.9ms
Speed: 7.2ms preprocess, 270.9ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)



...

[     327.28      371.45      374.38      408.51         169]
[     348.66      316.35      388.73      362.86         165]
[     260.81      309.63      316.07      367.76         153]
[     301.95      307.86      334.16      335.74         172]
[     321.86       383.2      369.45      419.05         169]
[     344.48       323.7      385.98      373.31         165]
[     252.29      314.25      310.57      376.76         153]


0: 384x640 7 cars, 1 truck, 155.0ms
Speed: 5.5ms preprocess, 155.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 bus, 2 trucks, 130.5ms
Speed: 3.5ms preprocess, 130.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



[      295.3      312.94      329.48      342.68         172]
[     338.47      331.62      382.91      384.94         165]
[     242.13      320.75      303.28      386.92         153]
[     370.63      276.99      402.83      307.89         173]
[     288.86      317.51      324.72      349.43         172]
[     333.94      277.45      363.65      301.94         168]
[     331.91      337.22      379.63      397.51         165]
[     231.05      327.36      295.63      396.79         153]


0: 384x640 7 cars, 178.2ms
Speed: 2.9ms preprocess, 178.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 152.2ms
Speed: 3.4ms preprocess, 152.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


[     323.36      285.28      362.49      313.84         175]
[     301.22      274.94      330.87      296.43         174]
[      282.2      324.18      319.54      357.43         172]
[     330.01       280.8      360.69       305.4         168]
[     325.49      344.41      375.27      409.26         165]
[     296.25      274.95       327.7      298.45         174]
[     274.68      330.15      314.32      365.81         172]
[     318.78      351.01      370.22      417.25         165]


# people counter -

In [ ]:
# here we will create counter for two lanes -

In [ ]:
cap = cv2.VideoCapture(r"C:\Users\Dell\Downloads\Analysis\project9-Object_detection\vid1.mp4") # path
model = YOLO(r'C:\Users\Dell\Downloads\Analysis\project9-Object_detection\yolov8n.pt')
mask = cv2.imread('xx.png')
# resize mask -
mask = cv2.resize(mask, (int(cap.get(3)), int(cap.get(4))))
tracker = Sort(max_age=20,min_hits=3,iou_threshold=0.3)
limitsUp = [103, 161, 296, 161]
limitsDown = [527, 489, 735, 489]
 
totalCountUp = []
totalCountDown = []
 
while True:
    success, img = cap.read()
    imgRegion = cv2.bitwise_and(img, mask)
 
    imgGraphics = cv2.imread("graphics.png", cv2.IMREAD_UNCHANGED)
    img = cvzone.overlayPNG(img, imgGraphics, (730, 260))
    results = model(imgRegion, stream=True)
 
    detections = np.empty((0, 5))
 
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
            w, h = x2 - x1, y2 - y1
 
            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Class Name
            cls = int(box.cls[0])
            currentClass = classNames[cls]
 
            if currentClass == "person" and conf > 0.3:
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))
 
    resultsTracker = tracker.update(detections)
 
    cv2.line(img, (limitsUp[0], limitsUp[1]), (limitsUp[2], limitsUp[3]), (0, 0, 255), 5)
    cv2.line(img, (limitsDown[0], limitsDown[1]), (limitsDown[2], limitsDown[3]), (0, 0, 255), 5)
 
    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        print(result)
        w, h = x2 - x1, y2 - y1
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
        cvzone.putTextRect(img, f' {int(id)}', (max(0, x1), max(35, y1)),
                           scale=2, thickness=3, offset=10)
 
        cx, cy = x1 + w // 2, y1 + h // 2
        cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)
 
        if limitsUp[0] < cx < limitsUp[2] and limitsUp[1] - 15 < cy < limitsUp[1] + 15:
            if totalCountUp.count(id) == 0:
                totalCountUp.append(id)
                cv2.line(img, (limitsUp[0], limitsUp[1]), (limitsUp[2], limitsUp[3]), (0, 255, 0), 5)
 
        if limitsDown[0] < cx < limitsDown[2] and limitsDown[1] - 15 < cy < limitsDown[1] + 15:
            if totalCountDown.count(id) == 0:
                totalCountDown.append(id)
                cv2.line(img, (limitsDown[0], limitsDown[1]), (limitsDown[2], limitsDown[3]), (0, 255, 0), 5)
    # # cvzone.putTextRect(img, f' Count: {len(totalCount)}', (50, 50))
    cv2.putText(img,str(len(totalCountUp)),(929,345),cv2.FONT_HERSHEY_PLAIN,5,(139,195,75),7)
    cv2.putText(img,str(len(totalCountDown)),(1191,345),cv2.FONT_HERSHEY_PLAIN,5,(50,50,230),7)
 
    cv2.imshow("Image", img)
    # cv2.imshow("ImageRegion", imgRegion)
    key = cv2.waitKey(1) & 0xFF
    if key != 255:  # Check if any key is pressed (not equal to 255)
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()